In [1]:
import wandb
from sklearn.model_selection import train_test_split
import os,cv2
import numpy as np
import matplotlib.pyplot as plt
from torch.nn import *
import torch,torchvision
from tqdm import tqdm
device = 'cuda'
PROJECT_NAME = 'MNIST-Clf'

In [2]:
def load_data():
    labels = {}
    labels_r = {}
    idx = -1
    data = []
    for folder in tqdm(os.listdir('./data/')):
        idx += 1
        labels[folder] = idx
        labels_r[idx] = folder
        for file in os.listdir(f'./data/{folder}')[:1000]:
            img = cv2.imread(f'./data/{folder}/{file}')
            img = cv2.resize(img,(112,112))
            img = img / 255.0
            data.append([img,idx])
    np.random.shuffle(data)
    np.random.shuffle(data)
    X = []
    y = []
    for d in data:
        X.append(d[0])
        y.append(d[1])
    X_train,X_test,y_train,y_test = train_test_split(X,y,shuffle=False,test_size=0.125)
    return X_train,X_test,y_train,y_test,X,y,labels,labels_r,idx,data

In [3]:
X_train,X_test,y_train,y_test,X,y,labels,labels_r,idx,data = load_data()

100%|███████████████████████████████████████████| 10/10 [00:01<00:00,  6.73it/s]


In [4]:
X_train = torch.from_numpy(np.array(X_train)).view(-1,3,112,112).to(device).float()
X_test = torch.from_numpy(np.array(X_test)).view(-1,3,112,112).to(device).float()
y_train = torch.from_numpy(np.array(y_train)).to(device)
y_test = torch.from_numpy(np.array(y_test)).to(device)

In [5]:
def get_loss(model,X,y,criterion):
    preds = model(X)
    loss = criterion(preds,y)
    return loss.item()

In [6]:
def get_accuracy(model,X,y):
    correct = 0
    total = 0
    preds = model(X)
    for pred,y_batch in zip(preds,y):
        pred = int(torch.argmax(pred))
        y_batch = int(torch.argmax(y_batch))
        if pred == y_batch:
            correct += 1
        total += 1
    acc = round(correct/total,3)*100
    return acc

In [7]:
def get_pred(model):
    model.eval()
    for file in os.listdir('./test_data/'):
        img = cv2.imread(f'./test_data/{file}')
        img = cv2.resize(img,(112,112)) / 255.0
        img = torch.from_numpy(np.array(img)).to(device)
        pred = model(img)
        plt.figure(figsize=(12,6))
        plt.imshow(img)
        plt.title(f'{labels_r[int(torch.argmax(pred))]}')
        plt.savefig(f'./preds/{file}')
        plt.close()
    model.train()

In [8]:
class Model(Module):
    def __init__(self):
        super().__init__()
        self.activation = ReLU()
        self.max_pool2d = MaxPool2d((2,2),(2,2))
        self.conv1 = Conv2d(3,6,(3,3))
        self.conv1batchnorm = BatchNorm2d(6)
        self.conv2 = Conv2d(6,9,(3,3))
        self.conv2batchnorm = BatchNorm2d(9)
        self.conv3 = Conv2d(9,12,(3,3))
        self.conv3batchnorm = BatchNorm2d(12)
        self.linear1 = Linear(12*12*12,128)
        self.linear1batchnorm = BatchNorm1d(128)
        self.linear2 = Linear(128,256)
        self.linear2batchnorm = BatchNorm1d(256)
        self.linear3 = Linear(256,128)
        self.linear3batchnorm = BatchNorm1d(128)
        self.output = Linear(128,len(labels))

    def forward(self,X):
        preds = self.activation(self.max_pool2d(self.conv1batchnorm(self.conv1(X))))
        preds = self.activation(self.max_pool2d(self.conv2batchnorm(self.conv2(preds))))
        preds = self.activation(self.max_pool2d(self.conv3batchnorm(self.conv3(preds))))
        preds = preds.view(-1,12*12*12)
        preds = self.activation(self.linear1batchnorm(self.linear1(preds)))
        preds = self.activation(self.linear2batchnorm(self.linear2(preds)))
        preds = self.activation(self.linear3batchnorm(self.linear3(preds)))
        preds = self.output(preds)
        return preds

In [9]:
model = Model().to(device)

In [10]:
criterion = CrossEntropyLoss()

In [11]:
from torch.optim import *

In [12]:
optimizer = Adam(model.parameters(),lr=0.001)

In [13]:
epochs = 100

In [14]:
batch_size = 32

In [ ]:
wandb.init(project=PROJECT_NAME,name='baseline')
wandb.watch(model)
for _ in tqdm(range(epochs)):
    for idx in range(0,len(X_train),batch_size):
        X_batch = X_train[idx:idx+batch_size].to(device).float().view(-1,3,112,112)
        y_batch = y_train[idx:idx+batch_size].to(device).long()
        preds = model(X_batch)
        loss = criterion(preds,y_batch)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    wandb.log({'Loss':get_loss(model,X_train,y_train,criterion)})
    wandb.log({'Val Loss':get_loss(model,X_test,y_test,criterion)})
    wandb.log({'Acc':get_accuracy(model,X_train,y_train)})
    wandb.log({'Val ACC':get_accuracy(model,X_test,y_test)})
    get_pred(model)
    for file in os.listdir('./preds/'):
        wandb.log({f'Img/{file}':wandb.Image(cv2.imread(f'./preds/{file}'))})
wandb.watch(model)
wandb.finish()